In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from utils_m6 import *
# sys.path.append('/data/stellaliu/loaddata/src')
# from utils import SQL, LoadData

In [2]:
return_vec = pd.read_csv('real_returns_0501.csv')
return_vec.shape

(1284, 100)

In [3]:
def calIR(p, returns):
    daily_returns = returns.T
    daily_returns['ID'] = daily_returns.index.tolist()
    decisions = pd.DataFrame({'decision': p, 'ID':daily_returns.index.tolist()})    
    portfolio = daily_returns.merge(decisions, on='ID')
    portfolio = portfolio[portfolio.decision != 0]
    weighted_portfolio_return = pd.DataFrame((portfolio.iloc[:, :daily_returns.shape[1]-1].to_numpy().T * portfolio.decision.to_numpy()).T)
    # print(weighted_portfolio_return)
    portfolio_daily_return = weighted_portfolio_return.apply(lambda x: np.log(1 + x)).sum(axis=0)
    # print(weighted_portfolio_return.sum(axis=0))
    # print(weighted_portfolio_return)

    ret_T = sum(portfolio_daily_return)
    D = portfolio_daily_return.shape[0]
    var = np.sum([(x - ret_T / D)**2 for x in portfolio_daily_return.tolist()]) / (D-1)
    sdp = np.sqrt(var)
    annualized_IR = (D+1)/D * 12 * ret_T / np.sqrt(252) / sdp
    return annualized_IR, portfolio_daily_return

def max_drawdown(price: np.array):
    # 最大回撤
    cum_max = np.maximum.accumulate(price)
    l = np.argmax(1 - price / cum_max)
    if l == 0: 
        return 0
    k = np.argmax(price[: l])
    return 1 - price[l] / price[k]

def normalize_weights(weight):
    weight = weight.squeeze()
    return weight/sum(abs(weight)+1e-5)

# optimal portfolio

In [ ]:
testing_periods = [int(i*22) for i in range(1, 25)]
training_duration = [int(i*22) for i in (1, 3, 6, 12)]
out = {}
for back_p in testing_periods:
    return_tmp = return_vec.iloc[:-back_p]
    
    for d in training_duration:
        if d not in out.keys():
            out[d] = [[], []]
        tmp = return_tmp.iloc[-d-22:-22]            
        test_period = return_tmp.iloc[-22:]
        try:
            weights, _, _ = optimal_portfolio(tmp.T.to_numpy())
            weights = normalize_weights(weights)
            IR, port_daily_rtn = calIR(weights, test_period)
            maxdd = max_drawdown(port_daily_rtn+1)
            out[d][0].append(IR)
            out[d][1].append(maxdd)
        except:
            out[d][0].append(np.nan)
            out[d][1].append(np.nan)

In [66]:
for d, values in out.items():
    print(d)
    print(np.nanmean(values[0]), np.nanstd(values[0]))
    print(np.nanmean(values[1]), np.nanstd(values[1]))
#     plt.plot(values[0], label=d)
# plt.legend()

22
0.21940833410128863 2.8776387218533306
0.11260254757613293 0.07195305408519459
66
1.0452936433347635 3.249533140614051
0.10872157600411396 0.06864715871621048
132
0.17591497964135958 3.07872468498706
0.10973131626191983 0.054448430427836765
264
0.40289834968842797 3.1211098493860314
0.11917656514074532 0.05204707194219107


# risk_parity

In [67]:
testing_periods = [int(i*22) for i in range(1, 25)]
training_duration = [int(i*22) for i in (1, 3, 6, 12)]
out = {}
for back_p in testing_periods:
    return_tmp = return_vec.iloc[:-back_p]
    
    for d in training_duration:
        if d not in out.keys():
            out[d] = [[], []]
        tmp = return_tmp.iloc[-d-22:-22]            
        test_period = return_tmp.iloc[-22:]
        try:
            weights = risk_parity(tmp.to_numpy(), 10)
            weights = normalize_weights(weights)
            IR, port_daily_rtn = calIR(weights, test_period)
            maxdd = max_drawdown(port_daily_rtn+1)
            out[d][0].append(IR)
            out[d][1].append(maxdd)
        except:
            out[d][0].append(np.nan)
            out[d][1].append(np.nan)

/data/stellaliu/m6/utils_m6.py:123: RuntimeWarning: invalid value encountered in sqrt
  step_size = 1 / np.sqrt(np.dot(np.dot(gradient, cov), gradient))
/home/ops/.pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


In [68]:
for d, values in out.items():
    print(d)
    print(np.nanmean(values[0]), np.nanstd(values[0]))
    print(np.nanmean(values[1]), np.nanstd(values[1]))
#     plt.plot(values[0], label=d)
# plt.legend()

22
-3.139467055585738 0.3145329960526051
0.0009018832230751284 0.0018495597115205317
66
-1.26612818167993 0.0
0.00031807015995706235 0.0009542104798711869
132
1.674811831867213 3.8954516334378435
0.0008776430598527285 0.0002959389132790759
264
1.5057202837505799 3.8439180770243815
0.0007008180317260027 0.00031602549192733914


# min_variance

In [75]:
testing_periods = [int(i*22) for i in range(1, 25)]
training_duration = [int(i*22) for i in (1, 3, 6, 12)]
out = {}
for back_p in testing_periods:
    return_tmp = return_vec.iloc[:-back_p]
    
    for d in training_duration:
        if d not in out.keys():
            out[d] = [[], []]
        tmp = return_tmp.iloc[-d-22:-22]            
        test_period = return_tmp.iloc[-22:]
        try:
            weights = min_variance(tmp.to_numpy())
            weights = normalize_weights(weights)
            IR, port_daily_rtn = calIR(weights, test_period)
            maxdd = max_drawdown(port_daily_rtn+1)
            out[d][0].append(IR)
            out[d][1].append(maxdd)
        except:
            out[d][0].append(np.nan)
            out[d][1].append(np.nan)

In [77]:
for d, values in out.items():
    print(d)
    print(np.nanmean(values[0]), np.nanstd(values[0]))
    print(np.nanmean(values[1]), np.nanstd(values[1]))
#     plt.plot(values[0], label=d)
# plt.legend()

22
1.1011590441284258 4.546310786760335
0.0031526390291338633 0.0012241656030521535
66
-1.8103245847961111 2.9647570345721626
0.0023357674112789327 0.0007260625091681702
132
-1.9442100516016132 4.226409628536494
0.0013615677506914835 0.0007666349246455914
264
-1.9661365367005605 3.012088187205887
0.0012367641417310726 0.0007617447269555287


# monte_carlo

In [78]:
testing_periods = [int(i*22) for i in range(1, 25)]
training_duration = [int(i*22) for i in (1, 3, 6, 12)]
out = {}
for back_p in testing_periods:
    return_tmp = return_vec.iloc[:-back_p]
    
    for d in training_duration:
        if d not in out.keys():
            out[d] = [[], []]
        tmp = return_tmp.iloc[-d-22:-22]            
        test_period = return_tmp.iloc[-22:]
        try:
            weights = monte_carlo(tmp.to_numpy(), 10)
            weights = normalize_weights(weights)
            IR, port_daily_rtn = calIR(weights, test_period)
            maxdd = max_drawdown(port_daily_rtn+1)
            out[d][0].append(IR)
            out[d][1].append(maxdd)
        except:
            out[d][0].append(np.nan)
            out[d][1].append(np.nan)

/data/stellaliu/m6/utils_m6.py:161: RuntimeWarning: invalid value encountered in sqrt
  portfolio_risk = np.sqrt(weights.T @ cov @ weights)


In [79]:
for d, values in out.items():
    print(d)
    print(np.nanmean(values[0]), np.nanstd(values[0]))
    print(np.nanmean(values[1]), np.nanstd(values[1]))
#     plt.plot(values[0], label=d)
# plt.legend()

22
1.1011590441284258 4.546310786760335
0.0031526390291338633 0.0012241656030521535
66
-1.8103245847961111 2.9647570345721626
0.0023357674112789327 0.0007260625091681702
132
-1.9442100516016132 4.226409628536494
0.0013615677506914835 0.0007666349246455914
264
-1.9661365367005605 3.012088187205887
0.0012367641417310726 0.0007617447269555287


# DE

In [9]:
testing_periods = [int(i*22) for i in range(1, 25)]
training_duration = [int(i*22) for i in (1, 3, 6, 12)]
out = {}
for back_p in testing_periods:
    return_tmp = return_vec.iloc[:-back_p]
    
    for d in training_duration:
        if d not in out.keys():
            out[d] = [[], []]
        tmp = return_tmp.iloc[-d-22:-22]       
        tmp.reset_index(drop=True).to_csv('DE_data.csv', index=False)     
        test_period = return_tmp.iloc[-22:]
        try:
            de = DE_portfolio(constraint_eq=[], 
                        constraint_ueq=[# 小于等于0
                                        lambda x: sum([abs(i) for i in x]) - 1,
                                        lambda x: 0.25 - sum([abs(i) for i in x])])
            vals, gen_weights = de.run(n_dim=100, size_pop=20, max_iter=200)
            weights = gen_weights[vals[0]]
            weights = normalize_weights(weights)
            IR, port_daily_rtn = calIR(weights, test_period)
            maxdd = max_drawdown(port_daily_rtn+1)
            out[d][0].append(IR)
            out[d][1].append(maxdd)
        except:
            out[d][0].append(np.nan)
            out[d][1].append(np.nan)



In [ ]:
for d, values in out.items():
    print(d)
    print(np.nanmean(values[0]), np.nanstd(values[0]))
    print(np.nanmean(values[1]), np.nanstd(values[1]))
#     plt.plot(values[0], label=d)
# plt.legend()

In [ ]:
# fig = plt.figure(figsize=(12,8))
# plt.plot(np.cumsum(portfolio_daily_return1), label='1-120day')

In [ ]:
# weight 1 # 120 days
weights = np.array([-2.75284008e-02, -3.97754789e-02, -1.81212586e-03,  1.38094175e-03,
        9.42208301e-03, -1.71030168e-02,  1.14672286e-02,  1.55726318e-03,
       -3.12849643e-03,  4.74580914e-03, -6.76903781e-04, -5.37125404e-03,
        7.08804758e-03,  2.39142977e-02, -2.54941283e-03, -4.08005023e-02,
       -2.82818006e-03, -2.59571343e-02,  1.15509116e-02,  1.85404813e-02,
        1.18334505e-02,  1.64851894e-02,  5.48628333e-03,  3.86290769e-03,
       -8.53277039e-04, -2.29118173e-02,  6.57054106e-03, -6.70110033e-03,
       -2.02847188e-02,  6.01843312e-05, -2.34666344e-02,  5.32287092e-03,
       -4.53631700e-03,  7.74146358e-03, -1.11778693e-02,  6.67408414e-04,
        1.15827128e-02, -2.57321920e-03,  1.09807912e-02, -1.82337074e-02,
        2.48102752e-02,  1.08430909e-02,  6.16284025e-03,  7.82775090e-03,
       -6.28516225e-03,  8.13584838e-04, -6.53712403e-03,  1.06443943e-02,
       -1.89856483e-03,  1.29077186e-02, -8.07122487e-03, -8.09023021e-03,
       -8.19281864e-04,  2.49459948e-02,  1.93929268e-02, -4.28065101e-03,
       -2.28115902e-03,  4.61809807e-03, -1.06797195e-02, -7.99070309e-03,
        8.75342311e-03,  1.33069586e-02, -3.19995274e-02,  1.21457442e-02,
        2.72313067e-03,  3.59022548e-03,  1.12206126e-02,  2.07046774e-03,
        1.03556630e-02, -2.78444953e-03,  9.54520347e-03, -1.02724144e-02,
        1.09439228e-02, -3.74011540e-03,  1.96025619e-03,  7.79854629e-03,
        2.79722503e-03, -2.59017713e-03,  6.39016097e-03,  1.04569181e-03,
       -1.78378792e-02, -1.24232758e-02, -1.56128753e-03,  3.88042906e-03,
        8.52940550e-03,  2.98631458e-02,  2.19205801e-03,  9.89331743e-03,
       -1.70832663e-02, -1.52048052e-02, -1.86367488e-02, -7.27214163e-03,
       -1.32650271e-03,  9.52383313e-04,  5.92061054e-03, -8.91913593e-03,
        8.39285908e-03, -1.46319987e-02,  1.44360909e-02,  1.65778132e-02])
IR, _ = calIR(weights, test_period)
maxdd = max_drawdown(np.matmul(test_period.to_numpy(), weights))
print(f'{p} days, IR: {IR}, max drawdown: {maxdd}')

In [ ]:
# weight 3 120 days
weights = np.array([ 9.31499804e-03, -7.37171455e-03, -6.67439846e-03,  8.44276269e-04,
        2.56662719e-03, -9.12539339e-03, -3.96355150e-03, -3.68743637e-02,
        4.28443100e-03, -4.72926134e-02,  1.26031125e-02, -9.49158222e-03,
       -6.61660200e-03, -2.19049748e-03,  1.71918121e-02,  9.69876165e-03,
       -1.37785377e-02, -1.26571799e-02, -7.18187441e-04,  8.21603935e-03,
        1.50940904e-02,  4.70739599e-03,  2.11701460e-02, -1.66052022e-03,
        4.84288449e-03, -4.66635527e-02, -1.84184526e-03,  1.44503903e-02,
       -2.56952666e-02,  5.59681576e-03, -5.37003035e-04, -2.31629840e-02,
        1.14217672e-02, -1.30107508e-02,  2.14679518e-02, -2.67690341e-02,
       -8.01061911e-04,  1.10701100e-02,  3.53802162e-03,  1.79889539e-03,
        2.60833550e-03,  2.14545615e-03,  2.07638563e-03, -1.51920361e-02,
       -3.17571052e-03,  1.66234027e-04, -6.60655637e-03, -1.18813046e-02,
        7.95216867e-03,  4.91650831e-03, -1.15950570e-03,  1.30957203e-05,
       -1.35917819e-02, -2.26182914e-02,  1.20513636e-03, -5.09500605e-03,
       -2.56590248e-02, -2.78335941e-04,  2.09916389e-02, -2.22956781e-02,
       -1.56188366e-02, -1.18853119e-02,  1.10960798e-02,  5.54767568e-03,
        2.04977229e-03, -9.76411320e-03, -9.69276945e-03, -3.63078871e-03,
       -2.69552109e-03, -1.72121975e-02,  7.96733978e-04,  1.99369972e-02,
        9.48002615e-04, -3.97340976e-03,  7.88710713e-03, -1.86278078e-02,
        2.27533617e-02, -3.81913812e-03,  2.83952281e-03,  2.10727901e-02,
       -9.18262259e-03,  4.21778786e-03,  6.57816117e-03,  7.59234584e-04,
       -1.06660542e-03, -6.70237192e-03,  6.73810004e-03,  8.56616854e-03,
       -1.77705254e-02, -2.12258341e-02,  2.28216295e-02, -5.58229144e-03,
        9.20168471e-03,  1.59256963e-02, -5.64405703e-04,  7.83262536e-03,
       -8.65791085e-04, -3.80718574e-03, -8.04340957e-03,  4.29657328e-03])
IR, _ = calIR(weights, test_period)
maxdd = max_drawdown(np.matmul(test_period.to_numpy(), weights))
print(f'{p} days, IR: {IR}, max drawdown: {maxdd}')

In [ ]:
# weight 2 120 days
weights = np.array([-0.0011089 , -0.01534449, -0.00014131, -0.01356521, -0.01023588,
    -0.00048259, -0.00117397, -0.01942229,  0.0184835 ,  0.00242988,
     0.01469493, -0.00297221, -0.00818425, -0.0290448 , -0.00143429,
     0.01263725,  0.00011356, -0.00388848,  0.00057997,  0.00264273,
     0.01285719, -0.01697757,  0.00628504, -0.00406703, -0.00187891,
    -0.01076953, -0.01490095, -0.02187824, -0.01679134,  0.0001319 ,
    -0.02624278,  0.00911853, -0.03428521, -0.00511216, -0.01134546,
     0.01388879,  0.03199736,  0.00154435,  0.01015098,  0.01092993,
    -0.00770203, -0.00429865,  0.0091382 ,  0.01306989,  0.01358856,
     0.03160157,  0.00091563, -0.01285225,  0.00306426,  0.00205626,
     0.00283003, -0.00372706, -0.00475768,  0.00998402,  0.00329832,
     0.01625984, -0.00541464,  0.00656341, -0.00832898, -0.00286968,
    -0.00603483, -0.00202703, -0.01123155, -0.00815741,  0.02517895,
     0.00816442,  0.00505389, -0.00882266, -0.0089342 , -0.00077233,
    -0.0148778 , -0.00458401, -0.01417049, -0.00731738, -0.02718698,
    -0.02294586, -0.01707459, -0.0240783 , -0.00316166, -0.00066705,
    -0.00538873,  0.00797266, -0.01348659, -0.00301472, -0.00165216,
    -0.00019332,  0.00381956,  0.00853446, -0.00869554, -0.02826617,
     0.00795792,  0.00089001, -0.0149529 , -0.02240482, -0.00874062,
    -0.01254003,  0.0023372 , -0.01921658, -0.01112407,  0.01631586])

IR, _ = calIR(weights, test_period)
maxdd = max_drawdown(np.matmul(test_period.to_numpy(), weights))
print(f'{p} days, IR: {IR}, max drawdown: {maxdd}')

In [ ]:
# weight 2 120 days
weights = np.array([ 0.00757067, -0.01198137,  0.01127877, -0.01497662, -0.01281749,
       -0.00485938,  0.00182533,  0.00181949, -0.00463318, -0.00841613,
        0.00562399, -0.01236804,  0.00046793,  0.00506014, -0.02024252,
        0.03796806, -0.00478788, -0.01917832,  0.00545143, -0.00937888,
        0.01993783, -0.00150363, -0.01959124, -0.00987594,  0.01503409,
        0.0143857 ,  0.00409182, -0.00562504,  0.01186271, -0.00355423,
        0.00803681, -0.01234237,  0.00015623, -0.00214829, -0.00506922,
       -0.01871814, -0.00392203, -0.00389231,  0.03669328,  0.00563558,
        0.00460333,  0.00540384,  0.00424547,  0.00496614, -0.00583057,
        0.02502617,  0.00660983, -0.01219363, -0.02164242, -0.0087888 ,
        0.00115127,  0.00362848,  0.02045804,  0.00050135, -0.01082015,
       -0.00991893,  0.00864505, -0.02349094,  0.00072248, -0.01324389,
        0.01142244,  0.02669029, -0.00243466,  0.00721455, -0.00130876,
        0.01400447, -0.00215836, -0.00267733, -0.0061375 ,  0.01431903,
        0.01493543,  0.00864569, -0.02024223, -0.01947277, -0.00337026,
        0.0078421 , -0.01440679,  0.00245046,  0.00012015,  0.01721802,
        0.01924505, -0.00139176, -0.02152844,  0.0033472 ,  0.02746866,
        0.02008293,  0.01173267,  0.0013667 ,  0.00171248, -0.00583026,
       -0.00126022, -0.0030769 ,  0.01003167,  0.00571254,  0.01946232,
       -0.00442757, -0.00712734,  0.00898744, -0.01339428, -0.02106937])
IR, _ = calIR(weights, test_period)
maxdd = max_drawdown(np.matmul(test_period.to_numpy(), weights))
print(f'{p} days, IR: {IR}, max drawdown: {maxdd}')

In [ ]:
# weight 2 120 days
weights = np.array([ 0.0071794 , -0.01515677,  0.01069585, -0.01420257, -0.01215504,
        0.02638502,  0.0041905 ,  0.00172545, -0.00439372, -0.00798115,
        0.00533332,  0.00958057,  0.00044374,  0.00479862, -0.01919631,
        0.00808343, -0.00454043, -0.01818712,  0.00516968, -0.00889415,
        0.01890738, -0.00142591, -0.00967084, -0.00936552,  0.03665209,
       -0.01498565,  0.00388034, -0.00533432,  0.0112496 ,  0.02818324,
        0.0013203 , -0.01170448,  0.00014815, -0.02922813, -0.00480723,
       -0.01775072, -0.00371933, -0.0275698 , -0.0342275 ,  0.00534431,
       -0.00398368,  0.00512455,  0.00402605,  0.00470947, -0.00552923,
        0.00084374,  0.00626821,  0.00141662, -0.02052386, -0.00833456,
        0.00109176,  0.00344095,  0.0194007 ,  0.00047544, -0.01026092,
       -0.00940628,  0.00819825, -0.01114414,  0.0040099 , -0.0125594 ,
        0.01083209,  0.02531084, -0.00230883,  0.00684168, -0.00124112,
        0.01580353, -0.00204681,  0.0157553 , -0.01016762, -0.00085304,
        0.01416352,  0.00819886, -0.01919604, -0.01846635, -0.00319608,
        0.00743679,  0.01299287,  0.00232381, -0.00606529, -0.00149302,
        0.0182504 , -0.00131983, -0.02041577,  0.00317421,  0.02604899,
        0.01904497,  0.01112629,  0.00129606,  0.00162398, -0.013145  ,
       -0.00119509, -0.00291787,  0.01519584,  0.00541729,  0.01845644,
       -0.00419874, -0.00675898,  0.00852293, -0.01270202, -0.01998043])
IR, _ = calIR(weights, test_period)
maxdd = max_drawdown(np.matmul(test_period.to_numpy(), weights))
print(f'{p} days, IR: {IR}, max drawdown: {maxdd}')

In [44]:
periods = [22, 22*3, 22*6, 22*12]
for p in periods:
    tmp = return_vec.iloc[-p:]
    test_period = return_vec.iloc[-66:]
    weights = risk_parity(tmp.to_numpy(), 10)
    weights = normalize_weights(weights)
    IR, _ = calIR(weights, test_period)
    maxdd = max_drawdown(np.matmul(test_period.to_numpy(), weights))
    print(f'{p} days, IR: {IR}, max drawdown: {maxdd}')

22 days, IR: nan, max drawdown: 0
66 days, IR: nan, max drawdown: 0
132 days, IR: -21.456124350855067, max drawdown: 2.641887561309039
264 days, IR: -2.310119059632797, max drawdown: 2.372191271282624


/data/stellaliu/algo_trade_data_tasks/utils.py:123: RuntimeWarning: invalid value encountered in sqrt
  step_size = 1 / np.sqrt(np.dot(np.dot(gradient, cov), gradient))
/tmp/ipykernel_12870/2717258453.py:17: RuntimeWarning: invalid value encountered in scalar divide
  annualized_IR = (D+1)/D * 12 * ret_T / np.sqrt(252) / sdp
/data/stellaliu/algo_trade_data_tasks/utils.py:123: RuntimeWarning: invalid value encountered in sqrt
  step_size = 1 / np.sqrt(np.dot(np.dot(gradient, cov), gradient))
/tmp/ipykernel_12870/2717258453.py:17: RuntimeWarning: invalid value encountered in scalar divide
  annualized_IR = (D+1)/D * 12 * ret_T / np.sqrt(252) / sdp


In [46]:
periods = [22, 22*3, 22*6, 22*12]
for p in periods:
    tmp = return_vec.iloc[-p:]
    test_period = return_vec.iloc[-66:]
    weights = min_variance(tmp.to_numpy())
    weights = normalize_weights(weights)
    IR, _ = calIR(weights, test_period)
    maxdd = max_drawdown(np.matmul(test_period.to_numpy(), weights))
    print(f'{p} days, IR: {IR}, max drawdown: {maxdd}')

22 days, IR: 6.247216807816301, max drawdown: 2.4528048664721447
66 days, IR: -4275.465375716355, max drawdown: 0
132 days, IR: -26.265665229155747, max drawdown: 8.097759832265737
264 days, IR: -12.640535138787566, max drawdown: 8.834762452214004


In [50]:
periods = [22, 22*3, 22*6, 22*12]
for p in periods:
    tmp = return_vec.iloc[-p:]
    test_period = return_vec.iloc[-66:]
    weights = monte_carlo(tmp.to_numpy(), 10)
    weights = normalize_weights(weights)
    IR, _ = calIR(weights, test_period)
    maxdd = max_drawdown(np.matmul(test_period.to_numpy(), weights))
    print(f'{p} days, IR: {IR}, max drawdown: {maxdd}')

22 days, IR: 6.247216807816301, max drawdown: 2.4528048664721447
66 days, IR: -4275.465375716355, max drawdown: 0
132 days, IR: -26.265665229155747, max drawdown: 8.097759832265737
264 days, IR: -12.640535138787566, max drawdown: 8.834762452214004


/data/stellaliu/algo_trade_data_tasks/utils.py:161: RuntimeWarning: invalid value encountered in sqrt
  portfolio_risk = np.sqrt(weights.T @ cov @ weights)


In [9]:
weights = monte_carlo(return_vec.to_numpy(), 20)
weights = normalize_weights(weights)
IR, portfolio_daily_return1 = calIR(weights, return_vec)
IR

53.94527446743518

In [10]:
de = DE_portfolio(constraint_eq=[], 
                            constraint_ueq=[
                                            # 小于等于0
                                            lambda x: sum([abs(i) for i in x]) - 1,
                                            lambda x: 0.25 - sum([abs(i) for i in x])
                                        ])

vals, gen_weights = de.run()

best_y: [10.70252467]


In [11]:
vals

array([13, 12, 16, 19, 18, 17, 11, 10, 15, 14,  9,  8,  7,  6,  5,  4,  2,
        1,  3,  0])

In [29]:
# fig = plt.figure(figsize=(12,8))
# plt.plot(np.cumsum(portfolio_daily_return1), label='1-120day')

In [54]:
# weight 1 # 120 days
weights = np.array([-2.75284008e-02, -3.97754789e-02, -1.81212586e-03,  1.38094175e-03,
        9.42208301e-03, -1.71030168e-02,  1.14672286e-02,  1.55726318e-03,
       -3.12849643e-03,  4.74580914e-03, -6.76903781e-04, -5.37125404e-03,
        7.08804758e-03,  2.39142977e-02, -2.54941283e-03, -4.08005023e-02,
       -2.82818006e-03, -2.59571343e-02,  1.15509116e-02,  1.85404813e-02,
        1.18334505e-02,  1.64851894e-02,  5.48628333e-03,  3.86290769e-03,
       -8.53277039e-04, -2.29118173e-02,  6.57054106e-03, -6.70110033e-03,
       -2.02847188e-02,  6.01843312e-05, -2.34666344e-02,  5.32287092e-03,
       -4.53631700e-03,  7.74146358e-03, -1.11778693e-02,  6.67408414e-04,
        1.15827128e-02, -2.57321920e-03,  1.09807912e-02, -1.82337074e-02,
        2.48102752e-02,  1.08430909e-02,  6.16284025e-03,  7.82775090e-03,
       -6.28516225e-03,  8.13584838e-04, -6.53712403e-03,  1.06443943e-02,
       -1.89856483e-03,  1.29077186e-02, -8.07122487e-03, -8.09023021e-03,
       -8.19281864e-04,  2.49459948e-02,  1.93929268e-02, -4.28065101e-03,
       -2.28115902e-03,  4.61809807e-03, -1.06797195e-02, -7.99070309e-03,
        8.75342311e-03,  1.33069586e-02, -3.19995274e-02,  1.21457442e-02,
        2.72313067e-03,  3.59022548e-03,  1.12206126e-02,  2.07046774e-03,
        1.03556630e-02, -2.78444953e-03,  9.54520347e-03, -1.02724144e-02,
        1.09439228e-02, -3.74011540e-03,  1.96025619e-03,  7.79854629e-03,
        2.79722503e-03, -2.59017713e-03,  6.39016097e-03,  1.04569181e-03,
       -1.78378792e-02, -1.24232758e-02, -1.56128753e-03,  3.88042906e-03,
        8.52940550e-03,  2.98631458e-02,  2.19205801e-03,  9.89331743e-03,
       -1.70832663e-02, -1.52048052e-02, -1.86367488e-02, -7.27214163e-03,
       -1.32650271e-03,  9.52383313e-04,  5.92061054e-03, -8.91913593e-03,
        8.39285908e-03, -1.46319987e-02,  1.44360909e-02,  1.65778132e-02])
IR, _ = calIR(weights, test_period)
maxdd = max_drawdown(np.matmul(test_period.to_numpy(), weights))
print(f'{p} days, IR: {IR}, max drawdown: {maxdd}')

264 days, IR: 10.30808696288808, max drawdown: 2.156211335159785


In [55]:
# weight 3 120 days
weights = np.array([ 9.31499804e-03, -7.37171455e-03, -6.67439846e-03,  8.44276269e-04,
        2.56662719e-03, -9.12539339e-03, -3.96355150e-03, -3.68743637e-02,
        4.28443100e-03, -4.72926134e-02,  1.26031125e-02, -9.49158222e-03,
       -6.61660200e-03, -2.19049748e-03,  1.71918121e-02,  9.69876165e-03,
       -1.37785377e-02, -1.26571799e-02, -7.18187441e-04,  8.21603935e-03,
        1.50940904e-02,  4.70739599e-03,  2.11701460e-02, -1.66052022e-03,
        4.84288449e-03, -4.66635527e-02, -1.84184526e-03,  1.44503903e-02,
       -2.56952666e-02,  5.59681576e-03, -5.37003035e-04, -2.31629840e-02,
        1.14217672e-02, -1.30107508e-02,  2.14679518e-02, -2.67690341e-02,
       -8.01061911e-04,  1.10701100e-02,  3.53802162e-03,  1.79889539e-03,
        2.60833550e-03,  2.14545615e-03,  2.07638563e-03, -1.51920361e-02,
       -3.17571052e-03,  1.66234027e-04, -6.60655637e-03, -1.18813046e-02,
        7.95216867e-03,  4.91650831e-03, -1.15950570e-03,  1.30957203e-05,
       -1.35917819e-02, -2.26182914e-02,  1.20513636e-03, -5.09500605e-03,
       -2.56590248e-02, -2.78335941e-04,  2.09916389e-02, -2.22956781e-02,
       -1.56188366e-02, -1.18853119e-02,  1.10960798e-02,  5.54767568e-03,
        2.04977229e-03, -9.76411320e-03, -9.69276945e-03, -3.63078871e-03,
       -2.69552109e-03, -1.72121975e-02,  7.96733978e-04,  1.99369972e-02,
        9.48002615e-04, -3.97340976e-03,  7.88710713e-03, -1.86278078e-02,
        2.27533617e-02, -3.81913812e-03,  2.83952281e-03,  2.10727901e-02,
       -9.18262259e-03,  4.21778786e-03,  6.57816117e-03,  7.59234584e-04,
       -1.06660542e-03, -6.70237192e-03,  6.73810004e-03,  8.56616854e-03,
       -1.77705254e-02, -2.12258341e-02,  2.28216295e-02, -5.58229144e-03,
        9.20168471e-03,  1.59256963e-02, -5.64405703e-04,  7.83262536e-03,
       -8.65791085e-04, -3.80718574e-03, -8.04340957e-03,  4.29657328e-03])
IR, _ = calIR(weights, test_period)
maxdd = max_drawdown(np.matmul(test_period.to_numpy(), weights))
print(f'{p} days, IR: {IR}, max drawdown: {maxdd}')

264 days, IR: 4.738690219762492, max drawdown: 1.8661304043669216


In [57]:
# weight 2 120 days
weights = np.array([-0.0011089 , -0.01534449, -0.00014131, -0.01356521, -0.01023588,
    -0.00048259, -0.00117397, -0.01942229,  0.0184835 ,  0.00242988,
     0.01469493, -0.00297221, -0.00818425, -0.0290448 , -0.00143429,
     0.01263725,  0.00011356, -0.00388848,  0.00057997,  0.00264273,
     0.01285719, -0.01697757,  0.00628504, -0.00406703, -0.00187891,
    -0.01076953, -0.01490095, -0.02187824, -0.01679134,  0.0001319 ,
    -0.02624278,  0.00911853, -0.03428521, -0.00511216, -0.01134546,
     0.01388879,  0.03199736,  0.00154435,  0.01015098,  0.01092993,
    -0.00770203, -0.00429865,  0.0091382 ,  0.01306989,  0.01358856,
     0.03160157,  0.00091563, -0.01285225,  0.00306426,  0.00205626,
     0.00283003, -0.00372706, -0.00475768,  0.00998402,  0.00329832,
     0.01625984, -0.00541464,  0.00656341, -0.00832898, -0.00286968,
    -0.00603483, -0.00202703, -0.01123155, -0.00815741,  0.02517895,
     0.00816442,  0.00505389, -0.00882266, -0.0089342 , -0.00077233,
    -0.0148778 , -0.00458401, -0.01417049, -0.00731738, -0.02718698,
    -0.02294586, -0.01707459, -0.0240783 , -0.00316166, -0.00066705,
    -0.00538873,  0.00797266, -0.01348659, -0.00301472, -0.00165216,
    -0.00019332,  0.00381956,  0.00853446, -0.00869554, -0.02826617,
     0.00795792,  0.00089001, -0.0149529 , -0.02240482, -0.00874062,
    -0.01254003,  0.0023372 , -0.01921658, -0.01112407,  0.01631586])

IR, _ = calIR(weights, test_period)
maxdd = max_drawdown(np.matmul(test_period.to_numpy(), weights))
print(f'{p} days, IR: {IR}, max drawdown: {maxdd}')

264 days, IR: 1.3596384066679554, max drawdown: 2.2756551915246073


In [58]:
# weight 2 120 days
weights = np.array([ 0.00757067, -0.01198137,  0.01127877, -0.01497662, -0.01281749,
       -0.00485938,  0.00182533,  0.00181949, -0.00463318, -0.00841613,
        0.00562399, -0.01236804,  0.00046793,  0.00506014, -0.02024252,
        0.03796806, -0.00478788, -0.01917832,  0.00545143, -0.00937888,
        0.01993783, -0.00150363, -0.01959124, -0.00987594,  0.01503409,
        0.0143857 ,  0.00409182, -0.00562504,  0.01186271, -0.00355423,
        0.00803681, -0.01234237,  0.00015623, -0.00214829, -0.00506922,
       -0.01871814, -0.00392203, -0.00389231,  0.03669328,  0.00563558,
        0.00460333,  0.00540384,  0.00424547,  0.00496614, -0.00583057,
        0.02502617,  0.00660983, -0.01219363, -0.02164242, -0.0087888 ,
        0.00115127,  0.00362848,  0.02045804,  0.00050135, -0.01082015,
       -0.00991893,  0.00864505, -0.02349094,  0.00072248, -0.01324389,
        0.01142244,  0.02669029, -0.00243466,  0.00721455, -0.00130876,
        0.01400447, -0.00215836, -0.00267733, -0.0061375 ,  0.01431903,
        0.01493543,  0.00864569, -0.02024223, -0.01947277, -0.00337026,
        0.0078421 , -0.01440679,  0.00245046,  0.00012015,  0.01721802,
        0.01924505, -0.00139176, -0.02152844,  0.0033472 ,  0.02746866,
        0.02008293,  0.01173267,  0.0013667 ,  0.00171248, -0.00583026,
       -0.00126022, -0.0030769 ,  0.01003167,  0.00571254,  0.01946232,
       -0.00442757, -0.00712734,  0.00898744, -0.01339428, -0.02106937])
IR, _ = calIR(weights, test_period)
maxdd = max_drawdown(np.matmul(test_period.to_numpy(), weights))
print(f'{p} days, IR: {IR}, max drawdown: {maxdd}')

264 days, IR: 12.26068356931353, max drawdown: 1.8084888512213282


In [59]:
# weight 2 120 days
weights = np.array([ 0.0071794 , -0.01515677,  0.01069585, -0.01420257, -0.01215504,
        0.02638502,  0.0041905 ,  0.00172545, -0.00439372, -0.00798115,
        0.00533332,  0.00958057,  0.00044374,  0.00479862, -0.01919631,
        0.00808343, -0.00454043, -0.01818712,  0.00516968, -0.00889415,
        0.01890738, -0.00142591, -0.00967084, -0.00936552,  0.03665209,
       -0.01498565,  0.00388034, -0.00533432,  0.0112496 ,  0.02818324,
        0.0013203 , -0.01170448,  0.00014815, -0.02922813, -0.00480723,
       -0.01775072, -0.00371933, -0.0275698 , -0.0342275 ,  0.00534431,
       -0.00398368,  0.00512455,  0.00402605,  0.00470947, -0.00552923,
        0.00084374,  0.00626821,  0.00141662, -0.02052386, -0.00833456,
        0.00109176,  0.00344095,  0.0194007 ,  0.00047544, -0.01026092,
       -0.00940628,  0.00819825, -0.01114414,  0.0040099 , -0.0125594 ,
        0.01083209,  0.02531084, -0.00230883,  0.00684168, -0.00124112,
        0.01580353, -0.00204681,  0.0157553 , -0.01016762, -0.00085304,
        0.01416352,  0.00819886, -0.01919604, -0.01846635, -0.00319608,
        0.00743679,  0.01299287,  0.00232381, -0.00606529, -0.00149302,
        0.0182504 , -0.00131983, -0.02041577,  0.00317421,  0.02604899,
        0.01904497,  0.01112629,  0.00129606,  0.00162398, -0.013145  ,
       -0.00119509, -0.00291787,  0.01519584,  0.00541729,  0.01845644,
       -0.00419874, -0.00675898,  0.00852293, -0.01270202, -0.01998043])
IR, _ = calIR(weights, test_period)
maxdd = max_drawdown(np.matmul(test_period.to_numpy(), weights))
print(f'{p} days, IR: {IR}, max drawdown: {maxdd}')

264 days, IR: 12.185074045132325, max drawdown: 1.8419784764750142
